<a href="https://colab.research.google.com/github/Neilus03/Restb.ai_challenge-HackUPC2023/blob/Neil_branch/TABNET2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_tabnet

In [ ]:


import json
import pickle
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error


In [ ]:

# Load data
Big_Data = pd.read_pickle("/content/drive/MyDrive/HackUPC/HackUPC_Restb.ia/hackupc2023_restbai.pkl")
Big_Data = Big_Data.T


In [ ]:

# Data preprocessing
city2number = {key: indice for indice, key in enumerate(Big_Data["city"].unique())}
neighborhood2number = {key: indice for indice, key in enumerate(Big_Data["neighborhood"].unique())}
region2number = {key: indice for indice, key in enumerate(Big_Data["region"].unique())}

Big_Data['city'] = Big_Data['city'].map(city2number)
Big_Data['neighborhood'] = Big_Data['neighborhood'].map(neighborhood2number)
Big_Data['region'] = Big_Data['region'].map(region2number)

Big_Data = Big_Data.drop(columns = ["summary", "images", "image_data", "property_type", "num_images"])
Big_Data = Big_Data.fillna(0)



In [ ]:

# Split data
train_data, valid_data = train_test_split(Big_Data, test_size=0.2, random_state=42)

# Normalize data
train_mean = train_data.mean()
train_std = train_data.std()

train_data = (train_data - train_mean) / train_std
valid_data = (valid_data - train_mean) / train_std

# Prepare data for TabNet
features = [col for col in train_data.columns if col != 'price']
target = ['price']

X_train = train_data[features].values
y_train = train_data[target].values.reshape(-1, 1)
X_valid = valid_data[features].values
y_valid = valid_data[target].values.reshape(-1, 1)



In [ ]:

# Initialize TabNetRegressor
model = TabNetRegressor()

# Fit model
model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['rmse'],
    max_epochs=100,
    patience=10,
    batch_size=256, 
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)





In [ ]:
# Predict on validation set
y_pred = model.predict(X_valid)

In [ ]:
# Compute RMSE
rmse = mean_squared_error(y_valid, y_pred, squared=False)
print(f'Validation RMSE: {rmse}')


In [ ]:

# Let's take a specific instance from the validation set to see the prediction
instance_index = 10
single_instance = X_valid[instance_index]
single_true_target = y_valid[instance_index]

# Making prediction
single_pred_target = model